In [1]:
import pandas as pd
import numpy as np
import pickle

# from crossref.restful import Works, Etiquette
# from scihub_blastoise import SciHub_HydroPump


### PART 1: Partial Run SumCetCet Analysis

We're back after Barça!

- The first order of business is to combine all partial fill SumCetCet run data to a single dataset to analyze.


In [7]:
# Sum:

with open("sum_pr_cr_results", "rb") as fp:
    sum_pr_cr_results = pickle.load(fp)

with open("sum_pr_sh_succ", "rb") as fp:
    sum_pr_sh_succ = pickle.load(fp)

with open("sum_pr_sh_fail", "rb") as fp:
    sum_pr_sh_fail = pickle.load(fp)

In [3]:
# Cet

with open("cet_pr_cr_results", "rb") as fp:
    cet_pr_cr_results = pickle.load(fp)

with open("cet_pr_sh_succ", "rb") as fp:
    cet_pr_sh_succ = pickle.load(fp)

with open("cet_pr_sh_fail", "rb") as fp:
    cet_pr_sh_fail = pickle.load(fp)

In [4]:
# Ogi:

with open("pr_cr_results", "rb") as fp:
    ogi_pr_cr_results = pickle.load(fp)

with open("pr_sh_succ", "rb") as fp:
    ogi_pr_sh_succ = pickle.load(fp)

with open("pr_sh_fail", "rb") as fp:
    ogi_pr_sh_fail = pickle.load(fp)

The initial analysis shows that around 50% of the metadata retr. from CR (pr_cr_results), was used for SH date extraction phase.

However, 40-50% of these articles actually contaion DATE info, therefore although the Req_Count is more than satisfied in the CR phase, SH results still do not satisfy for all journals.

Once the analysis is complete, we can use the pr_cr_results, to filter out the UNTESTED artcls, and pass them through the SH process.

In [46]:
sum_pr_cr_results = pd.DataFrame(sum_pr_cr_results)
sum_pr_sh_succ = pd.DataFrame(sum_pr_sh_succ)
sum_pr_sh_fail = pd.DataFrame(sum_pr_sh_fail)

sum_sh_results =  pd.concat([sum_pr_sh_succ, sum_pr_sh_fail])

sum_pr_results = pd.merge(sum_pr_cr_results, sum_sh_results, how="left", on="DOI")


In [47]:
cet_pr_cr_results = pd.DataFrame(cet_pr_cr_results)
cet_pr_sh_succ = pd.DataFrame(cet_pr_sh_succ)
cet_pr_sh_fail = pd.DataFrame(cet_pr_sh_fail)

cet_sh_results =  pd.concat([cet_pr_sh_succ, cet_pr_sh_fail])

cet_pr_results = pd.merge(cet_pr_cr_results, cet_sh_results, how="left", on="DOI")


In [48]:
ogi_pr_cr_results = pd.DataFrame(ogi_pr_cr_results)
ogi_pr_sh_succ = pd.DataFrame(ogi_pr_sh_succ)
ogi_pr_sh_fail = pd.DataFrame(ogi_pr_sh_fail)

ogi_sh_results =  pd.concat([ogi_pr_sh_succ, ogi_pr_sh_fail])

ogi_pr_results = pd.merge(ogi_pr_cr_results, ogi_sh_results, how="left", on="DOI")

Concatinating all partial run results to a single dataset: (pr_results_df)

In [54]:
pr_results_df = pd.concat([sum_pr_results, cet_pr_results,ogi_pr_results], ignore_index=True, verify_integrity=True)

In [59]:
pr_results_df.Dates.map(type).value_counts()

<class 'float'>    41840
<class 'str'>      35472
<class 'list'>     24708
Name: Dates, dtype: int64

Now that all the partial run results are consolidated:

TTD:
- Need to filter out the succesfull artcls, pass them through pre-processing, add them to q1_date_df

- Need to filter out FAILED, UNTESTED dfs


### PART 2: UPDATING q1_date_df

In [60]:
# TTD 1:
# Taken from 7c:

# 1.3 - Filtering to only successful artcls:
q1_date_pr = pr_results_df[pr_results_df.Dates.map(type)==list]
q1_date_pr.set_index("DOI", drop=True, inplace=True)

# 1.4 - Feature Eng. : 
def earlier_date_part(row):
    try:
        pp_year = row["published-print"]["date-parts"][0][0]
    except TypeError:
        pp_year = None

    try:   
        po_year = row["published-online"]["date-parts"][0][0]
    except TypeError:
        po_year = None

    if po_year is None:
        return row["published-print"]

    elif pp_year is None:
        return row["published-online"]

    else:
        if pp_year<po_year:
            return row["published-print"]
        elif po_year<pp_year:
            return row["published-online"]
        else:
            try:
                pp_month = row["published-print"]["date-parts"][0][1]
            except IndexError:
                pp_month = 12
            try:
                po_month = row["published-online"]["date-parts"][0][1]
            except IndexError:
                po_month = 12
            if pp_month<po_month:
                return row["published-print"]
            elif po_month<pp_month:
                return row["published-online"]
            else:
                return row["published-online"]


q1_date_pr.loc[:,"published"] = q1_date_pr.apply(earlier_date_part, axis=1)

q1_date_pr = q1_date_pr[["JRNL_ID", "ISSN", "issued", "container-title", "publisher", "title", "author", "subject", "references-count", "is-referenced-by-count", "published", "published-print", "published-online", "direct_url", "Dates", "Keywords"]]


<ipython-input-60-6f3939fe8a91>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q1_date_pr.loc[:,"published"] = q1_date_pr.apply(earlier_date_part, axis=1)


__q1_date_pr__ is the succesful articles from partial run dataset:

However, it should be noted that some (also apparent in q1_date_df) only have "Accepted Date" and not "Received Date"

This MUST be dealt with later in this notebook!!!


In [ ]:
with open("q1_date_df", "rb") as fp:
    q1_date_df = pickle.load(fp)

In [64]:
len(q1_date_df)

140993

In [65]:
q1_date_df = pd.concat([q1_date_df, q1_date_pr], verify_integrity=True)

__Fixing "only Accepted Date" issue:__

In [67]:
q1_date_df = q1_date_df[q1_date_df.Dates.map(lambda x: False if x[0]==None else True)]

SAVING & DUMPing all used datasets:

In [70]:
with open("q1_date_pr","wb") as p:
    pickle.dump(q1_date_pr, p)

In [121]:
# pr_results_df has ALL the extracted info from SumCetCet partial run

with open("pr_results_df","wb") as p:
    pickle.dump(pr_results_df, p)

In [69]:
with open("q1_date_df","wb") as p:
    pickle.dump(q1_date_df, p)

Currently q1_date_df has 156754 articles that are completely ready for analysis.

The remaining TDDs are:

__1- Article related:__
- Consolidate all UNTESTED article metadata -> only available from partial fill run dataset?
- Consolidate all FAIED articles to a single dataset -> final extraction, partial fill, q1_date_df_all, OTHERS?

__2- Journal related:__
- update q1_jrnl_df and q1_jrnl_expladed with the new information
- re-calc Req_Count & check if/how many UNTESTED can be used
- compare with pivots to check total necessary data

__3- Author related:__
- Finalize the author_df 
- Complete ScopusAPI Author Search run

__4- ANALYSIS & STATS & VISUALIZE__

__5- WRITE REPORT__

__6- FIN!__


### PART 3: UNTESTED AND FAILED Article Datasets

The datasets to be considered are:

- q1_date_df_all (Backups)
- q1_meta_fe
- q1_meta_fe_all
- date_df_full (5a)
- date_df_5b (5b)
- date_df_ok (Main Files)
- date_df_ok_final (Main Files)
- pr_results_df (7e)

To prevent to many variables each will be loaded & compared with q1_date_df and saved accordingly.


In [77]:
succ_doi_list = q1_date_df.index.tolist()

In [ ]:
# Code used for testing below:

# with open("date_df_ok_final", "rb") as fp:
#     testo = pickle.load(fp)

# temp_remn = testo[~testo.set_index("DOI").index.isin(succ_doi_list)].set_index("DOI")
# temp_remn = testo[~testo.index.isin(succ_doi_list)]

# remainder_df = pd.concat([remainder_df,q1_date_pr], verify_integrity=True)
# remainder_df = pd.concat([remainder_df,temp_remn])

__Results:__

- __q1_date_df_all (Backups)__
    - Total len: 36K
    - Not in q1_date_df: 4242
    - 

- __q1_meta_fe__
    - Total len: 138,184
    - Not in q1_date_df: 35,258
    - Already included : 131
    - 

- __q1_meta_fe_all__
    - Total len: 138,184
    - Not in q1_date_df: 35,258 
    - SKIPPED AS same as the previous df
    - 
- __date_df_full (5a)__
    - Total len: 128,374
    - Not in q1_date_df: 96,156
    - Already included: 4615
    - 
- __date_df_5b (5b)__
    - SKIPPED as pnly 36,514 articles with only SciHub information
    - 
- __date_df_ok (Main Files)__
    - Total len: 114,759
    - Not in q1_date_df: 82,686 
    - Already included: 82,773 SKIPPED as a result
- __date_df_ok_final (Main Files)__
    - Total len: 114,759
    - Not in q1_date_df: 82,686 
    - SKIPPED as same as previous
    - 
- __pr_results_df (7e)__
    - Total len:102,020
    - Not in q1_date_df: 78,645 
    - Already included: 4616
    - 


All the articles are concat'd into remainder_df. This df can be used to create FAILED & UNTESTED datasets

In [129]:
with open("remainder_df","wb") as p:
    pickle.dump(remainder_df, p)

#### PART 3.1: remainder_df:

In [2]:
with open("q1_date_df", "rb") as fp:
    q1_date_df = pickle.load(fp)

with open("remainder_df", "rb") as fp:
    remainder_df = pickle.load(fp)


In [5]:
remainder_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 214301 entries, 10.1007/s10441-011-9133-1 to 10.1214/18-ejp208
Data columns (total 21 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ISSN                    214301 non-null  object 
 1   issued                  214301 non-null  object 
 2   container-title         214301 non-null  object 
 3   publisher               214294 non-null  object 
 4   title                   214301 non-null  object 
 5   author                  192795 non-null  object 
 6   subject                 213933 non-null  object 
 7   references-count        118145 non-null  float64
 8   is-referenced-by-count  214301 non-null  int64  
 9   published               39116 non-null   object 
 10  published-print         201171 non-null  object 
 11  published-online        119803 non-null  object 
 12  direct_url              41047 non-null   object 
 13  Dates                   41047 non-null   obj

In [8]:
# Drop unnec. colms:
remainder_df.drop(["reference","language"], axis=1, inplace=True)

In [19]:
remainder_df[remainder_df.Results.notna()].Results.map(type).value_counts()

<class 'list'>    82686
<class 'str'>     13470
Name: Results, dtype: int64

In [22]:
remainder_df[(remainder_df.Results.notna() & (remainder_df.Results.map(type)==list))]["Results"].map(len).value_counts()

2    78240
3     4446
Name: Results, dtype: int64

In [13]:
remainder_df[(remainder_df.Dates.notna()) & (remainder_df.Results.notna()) ]

,ISSN,issued,container-title,publisher,title,author,subject,references-count,is-referenced-by-count,published,published-print,published-online,direct_url,Dates,Keywords,Affiliation,JRNL_ID,reference-count,Results
DOI,,,,,,,,,,,,,,,,,,,


In [12]:
remainder_df[(remainder_df["reference-count"].notna()) & (remainder_df["references-count"].notna()) ]

,ISSN,issued,container-title,publisher,title,author,subject,references-count,is-referenced-by-count,published,published-print,published-online,direct_url,Dates,Keywords,Affiliation,JRNL_ID,reference-count,Results
DOI,,,,,,,,,,,,,,,,,,,


__remainder_df__

__1- Succ:__
- has Dates & type == list
- has Results & len == 3

__2- Fail:__
- has Dates & type==str
- has Results & len==2

__3- Untested:__
- no Results & Dates cols.
- DOI not in any other df

In [24]:
# remainder_succ:

remainder_succ = remainder_df[(remainder_df.Dates.notna() & (remainder_df.Dates.map(type)==list))]

remainder_succ = pd.concat([remainder_succ, remainder_df.fillna("-")[(remainder_df.fillna("-").Results.notna() & (remainder_df.Results.fillna("-").map(len)==3))]])

In [33]:
# remainder_fail:

remainder_fail = remainder_df[(remainder_df.Dates.notna() & (remainder_df.Dates.map(type)==str))]

remainder_fail = pd.concat([remainder_fail, remainder_df.fillna("-")[(remainder_df.fillna("-").Results.notna()) & (remainder_df.Results.fillna("-").map(len)==2)]])

In [42]:
# remainder_untested:

# Need to drop all DOIs from other datasets

date_df_dois = q1_date_df.index.to_list()

rem_suc_dois = remainder_succ.index.tolist()

rem_fail_dois = remainder_fail.index.tolist()

In [43]:
all_dois = set(date_df_dois + rem_suc_dois + rem_fail_dois)

In [45]:
remainder_untested = remainder_df[~remainder_df.index.isin(all_dois)]

In [69]:
remainder_untested.shape

(80243, 19)

In [52]:
remainder_untested.Results.value_counts()

article_not_in_Scihub    9454
skipped                  3655
cant_read_pdf             242
direct_url_error           10
pdf_bytes_error             7
article_not_in Scihub       2
Name: Results, dtype: int64

In [55]:
remainder_untested[remainder_untested.Results.isna()].shape

(76588, 19)

The result above shows that remainder_untested has 89,958 articles remaining. However, due to the filtering conditions some are actually failed articles & will be moved to remainder_fail dataset.

The rest can be used in the following steps if necessary.


It is also seen that this step increased the number of untested artcl. count from 41K (pr_results_df) to 80K.


In [61]:
remainder_fail = pd.concat([remainder_fail, remainder_untested[~(remainder_untested.Results.isna()) | (remainder_untested.Results == "skipped")]])

remainder_untested = remainder_untested[(remainder_untested.Results.isna()) | (remainder_untested.Results == "skipped")]

All the remainder datasets are ready for future steps, so each will be saved accordingly.

In [70]:
with open("remainder_df","wb") as p:
    pickle.dump(remainder_df, p)

with open("remainder_succ","wb") as p:
    pickle.dump(remainder_succ, p)

with open("remainder_fail","wb") as p:
    pickle.dump(remainder_fail, p)

with open("remainder_untested","wb") as p:
    pickle.dump(remainder_untested, p)